In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

from IPython.display import clear_output
from datetime import datetime

# = = = = = = = = = = = = = = =

import os
import shutil

if os.path.exists('./file/3.oem'):
    shutil.rmtree('./file/3.oem')

os.makedirs('./file/3.oem')

if os.path.exists('./file/3.vehicle'):
    shutil.rmtree('./file/3.vehicle')

os.makedirs('./file/3.vehicle')

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

list_file = sorted(list(os.walk('./file/2.part'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

# = = = = = = = = = = = = = = =

for file in list_file:
    
    # = = = = = = = = = = = = = = =

    output_correct, output_correct_2, output_error = pd.DataFrame(), pd.DataFrame(), pd.DataFrame()

    # = = = = = = = = = = = = = = =

    input_ = pd.read_excel(f'./file/2.part/{file}',
                           header=0,
                           dtype=str).fillna('')
    
    input_['No'] = input_['No'].astype(int)
    input_['Rating Count'] = input_['Rating Count'].astype(int)

    print(f'总数量：{len(input_)}')
    print()

    # = = = = = = = = = = = = = = =


    work = Queue()
    for a in range(len(input_)):
        work.put_nowait(a)

    # = = = = = = = = = = = = = = =

    def crawler():
        global output_correct, output_correct_2, output_error

        while not work.empty():
            a = work.get_nowait()

            # = = = = = = = = = = = = = = =

            crawler_status = 'error'

            # = = = = = = = = = = = = = = =

            try:
                b = 0
                while True:
                    b += 1
                    
                    try:
                        request_url = input_.loc[a, 'OEM Url']
                        
                        resp = requests.get(request_url,
                                            headers=get_header(),
                                            proxies=get_proxy(),
                                            timeout=(10, 10))

                        if resp.status_code == 200:
                            soup = BeautifulSoup(resp.text, 'lxml')
                            html = etree.HTML(str(soup))
                            
                            break
                    except KeyboardInterrupt:
                        break
                    except:
                        continue

                # = = = = = = = = = = = = = = =

                list_th = [text.strip() for text in html.xpath('//table[@class="fitTable sortableTable onea-table"]/thead/tr/th/text()')]

                # = = = = = = = = = = = = = = =

                list_td = [[text.strip() for text in tr.xpath('./td/text()')] for tr in html.xpath('//table[@class="fitTable sortableTable onea-table"]/tbody/tr')]

                # = = = = = = = = = = = = = = =

                list_oem = [text.strip() for text in html.xpath('//table[@id="interchange-codes-table"]/tbody/tr/td[2]/text()')]
                oem = ';'.join(sorted(list(set(list_oem)), key=list_oem.index))

                # = = = = = = = = = = = = = = =

                df_temp = pd.DataFrame([{'No': input_.loc[a, 'No'],
                                         'Type': input_.loc[a, 'Type'],
                                         'SKU': input_.loc[a, 'SKU'],
                                         'Part Number': input_.loc[a, 'Part Number'],
                                         'Brand': input_.loc[a, 'Brand'],
                                         'Tag': input_.loc[a, 'Tag'],
                                         'Title': input_.loc[a, 'Title'],
                                         'OEM': oem,
                                         'Picture': input_.loc[a, 'Picture'],
                                         'Url': input_.loc[a, 'Url'],
                                         'Json_Src': input_.loc[a, 'Json_Src'],
                                         'OEM Url': input_.loc[a, 'OEM Url'],
                                         'Json_Description': input_.loc[a, 'Json_Description'],
                                         'Price': input_.loc[a, 'Price'],
                                         'Rating': input_.loc[a, 'Rating'],
                                         'Rating Count': input_.loc[a, 'Rating Count']}])
                
                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =

                df_temp = pd.DataFrame()
                for i, td in enumerate(list_td):
                    df_temp_temp = pd.DataFrame([{'No': input_.loc[a, 'No'],
                                                  'SKU': input_.loc[a, 'SKU'],
                                                  'OEM Url': input_.loc[a, 'OEM Url'],
                                                  'Vehicle No': i+1}])
                
                    for j, th in enumerate(list_th):
                        df_temp_temp.loc[0, th] = td[j]
                
                    year_begin = min([int(year) for year in df_temp_temp.loc[0, 'Year'].split('-')])
                    year_end = max([int(year) for year in df_temp_temp.loc[0, 'Year'].split('-')])
                    df_temp_temp.at[0, 'Year'] = [year for year in range(year_begin, year_end+1)]
                
                    df_temp_temp = df_temp_temp.explode('Year').reset_index(drop=True)
                    df_temp_temp['Year'] = df_temp_temp['Year'].astype(int)
                
                    df_temp = pd.concat([df_temp, df_temp_temp], ignore_index=True).fillna('')
                
                output_correct_2 = pd.concat([output_correct_2, df_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =

                crawler_status = 'ok'
            except:
                df_temp = pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)
                df_temp.loc[0, 'Request_Url'] = request_url
                
                output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

            # = = = = = = = = = = = = = = =

            if list_file.index(file) % 3 == 0 and list_file.index(file) != 0 and work.qsize() == 0 or work.qsize() % 1_000 == 0 and work.qsize() != 0:
                clear_output()

            print(f'''[状态：{crawler_status}，尝试次数：{b}] - {(list_file.index(file) + 1) / len(list_file) * 100:.2f}% - {file.removesuffix('.xlsx')} > {input_.loc[a, 'No']}.{request_url}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')

    # = = = = = = = = = = = = = = =

    list_task = []
    for _ in range(5):
        task = gevent.spawn(crawler)
        list_task.append(task)
    gevent.joinall(list_task)

    print('输出ing...')
    print()
    if not output_correct.empty:
        output_correct = output_correct.sort_values(by=['No'],
                                                    ascending=[True],
                                                    ignore_index=True)
        output_correct.to_excel(f'''./file/3.oem/{file.removesuffix('.xlsx')}-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
    if not output_correct_2.empty:
        output_correct_2 = output_correct_2.sort_values(by=['No', 'Vehicle No', 'Year'],
                                                        ascending=[True, True, False],
                                                        ignore_index=True)
        output_correct_2.to_excel(f'''./file/3.vehicle/{file.removesuffix('.xlsx')}-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
    if not output_error.empty:
        output_error = output_error.sort_values(by=['No'],
                                                ascending=[True],
                                                ignore_index=True)
        output_error.to_excel(f'''./file/{file.removesuffix('.xlsx')}-oem_error.xlsx''', index=False)
        print('爬虫存在error')
        print()

# = = = = = = = = = = = = = = =

list_file = sorted(list(os.walk('./file/3.oem'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

# = = = = = = = = = = = = = = =

from tqdm import tqdm
import re

for file in tqdm(list_file, desc='Progress', ncols=77):
    os.rename(f'./file/3.oem/{file}',
              f'''./file/3.oem/{re.sub(r'-[0-9]{8}_[0-9]{6}.xlsx$', '.xlsx', file)}''')

print('Done ~')

# = = = = = = = = = = = = = = =

list_file = sorted(list(os.walk('./file/3.vehicle'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

# = = = = = = = = = = = = = = =

for file in tqdm(list_file, desc='Progress', ncols=77):
    os.rename(f'./file/3.vehicle/{file}',
              f'''./file/3.vehicle/{re.sub(r'-[0-9]{8}_[0-9]{6}.xlsx$', '-vehicle.xlsx', file)}''')

print('Done ~')

总数量：10

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 5.https://www.1aauto.com/catalog/product/fit?skuId=1219510&pathName=%2F1996-00-dodge-chrysler-plymouth-front-ceramic-brake-pad-and-rotor-kit-with-calipers-and-hoses-trq-bka29570%2Fi%2F1abfs16142
[剩余数量：5] - [当前时间：14:44:42]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 2.https://www.1aauto.com/catalog/product/fit?skuId=1221644&pathName=%2F1996-00-toyota-4runner-with-16-inch-wheels-with-caliper-casting-%23s13wm-front-semi-metallic-performance-brake-pad-and-rotor-kit-with-calipers-and-hoses-trq-performance-bka34353%2Fi%2F1apbs07435
[剩余数量：4] - [当前时间：14:44:42]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 4.https://www.1aauto.com/catalog/product/fit?skuId=1194303&pathName=%2F1996-00-dodge-chrysler-plymouth-front-driver-and-passenger-side-ceramic-performance-brake-pad-and-rotor-kit-trq-performance-bka21616%2Fi%2F1apbs04134
[剩余数量：3] - [当前时间：14:44:42]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 3.https://www.1aauto.com/catalog/product/fit?skuId=1194192&pathName=%2F199

Progress: 100%|██████████████████████████████| 2/2 [00:00<00:00, 3782.06it/s]


Done ~


Progress: 100%|██████████████████████████████| 2/2 [00:00<00:00, 5633.72it/s]

Done ~
